In [96]:
import pandas as pd
import geopandas as gpd
import os
from geopy import distance
import calendar

os.chdir("/Users/anorawu/Documents/GitHub/CloudSeeding/data/炮台数据")

In [ ]:
fort_data = pd.read_csv("cleaned_炮台数据.csv")
fort_data = fort_data[fort_data['year']<=2022]

lon_max = fort_data['longitude'].max() + 1
lon_min = fort_data['longitude'].min() - 1
lat_max = fort_data['latitude'].max() + 1
lat_min = fort_data['latitude'].min() - 1

In [98]:
fort_points = fort_data[['index','longitude','latitude']].drop_duplicates()

In [99]:
rain_data = pd.read_stata("weatherstation_2010_2022.dta")
rain_data = rain_data.dropna(subset=['Lat', 'Lon'])
rain_data = rain_data[(rain_data['Lat'] >= lat_min) & (rain_data['Lat'] <= lat_max)]
rain_data = rain_data[(rain_data['Lon'] >= lon_min) & (rain_data['Lon'] <= lon_max)]

rain_points = rain_data[['StationId','Lon','Lat']].drop_duplicates()

In [100]:
rain_gdf = gpd.GeoDataFrame(
    rain_points,
    geometry=gpd.points_from_xy(rain_points['Lon'], rain_points['Lat']),
    crs="EPSG:4326" 
)

In [ ]:
fort_gdf = gpd.GeoDataFrame(
    fort_points,
    geometry=gpd.points_from_xy(fort_points['longitude'], fort_points['latitude']),
    crs="EPSG:4326" 
)

In [ ]:
radius = 0.5
dic = {}

for _, fort_point in fort_gdf.iterrows():  
    buffer = fort_point.geometry.buffer(radius)  
    
    pts_inside = rain_gdf[rain_gdf.within(buffer)]
    
    pts_dic = {}
    for _, row in pts_inside.iterrows():
        dist = distance.distance(
            (fort_point.geometry.y, fort_point.geometry.x),  
            (row['Lat'], row['Lon'])                        
        ).km
        pts_dic[row['StationId']] = dist
    
    sorted_pts = sorted(pts_dic.items(), key=lambda item: item[1])[:2]
    
    dic[fort_point['index']] = sorted_pts  



In [ ]:
columns = ['Alti', 'EVP', 'GST_Avg', 'GST_Max', 'GST_Min', 'PRE_Max_1h',
       'PRE_Time_2008', 'PRE_Time_0820', 'PRE_Time_2020', 'PRS_Avg', 'PRS_Max',
       'PRS_Min', 'RHU_Avg', 'RHU_Min', 'SSH', 'TEM_Avg', 'TEM_Max', 'TEM_Min',
       'WIN_S_2mi_Avg', 'WIN_S_10mi_Avg', 'WIN_D_S_Max', 'WIN_S_Max',
       'WIN_D_INST_Max', 'WIN_S_Inst_Max']

for year in range(2011,2023):
    for month in range(1,13):
        days_in_month = calendar.monthrange(year, month)[1]
        for day in range(1,days_in_month+1):
            for idx, fort_point in fort_points:
                pts_list = dic[fort_point['index']]

                weight_sum = 0
                list_of_data = []
                for pts in pts_list:
                    rain_id,dist = pts
                    # Make a copy of the filtered rows
                    filtered_row = rain_data[
                        (rain_data['Year'] == year) & 
                        (rain_data['Mon'] == month) & 
                        (rain_data['Day'] == day) & 
                        (rain_data['StationId'] == rain_id)
                    ].copy()  

                    filtered_row[columns] = filtered_row[columns] * 1 / (dist**2)
                    list_of_data.append(filtered_row)
                    weight_sum += 1/(dist**2) 
                
                concat_data = pd.concat(list_of_data)
                concat_data_sum = concat_data[columns].sum(min_count=4)
                    
                    

In [119]:
rain_data['EVP']

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
            ..
12032012   NaN
12032013   NaN
12032014   NaN
12032015   NaN
12032016   NaN
Name: EVP, Length: 11503308, dtype: float64

In [50]:
import pandas as pd
import numpy as np

# Generate a random DataFrame
np.random.seed(42)  # For reproducibility

# Create random data
n_rows = 10
data = {
    'Year': np.random.randint(2011, 2023, size=n_rows),
    'Mon': np.random.randint(1, 13, size=n_rows),
    'Day': np.random.randint(1, 29, size=n_rows),  # Assume 28 days max for simplicity
    'Lat': np.random.uniform(-90, 90, size=n_rows),
    'Lon': np.random.uniform(-180, 180, size=n_rows),
    'Rainfall': np.random.uniform(0, 100, size=n_rows),  # Random rainfall data
    'Temperature': np.random.uniform(-30, 50, size=n_rows),  # Random temperature data
}

# Create DataFrame
test = pd.DataFrame(data)
concat_test = pd.concat([test,test])
concat_test


,Year,Mon,Day,Lat,Lon,Rainfall,Temperature
0,2017,11,24,-88.728065,173.963119,23.089383,-13.364667
1,2014,8,12,-85.848763,-11.965358,24.102547,15.416026
2,2021,5,6,4.459439,129.578546,68.326352,-27.494937
3,2018,4,2,-18.025025,64.910714,60.999666,37.382782
4,2015,8,28,-81.600181,-17.820269,83.319491,5.980331
5,2017,8,21,85.275993,-175.224614,17.336465,1.612019
6,2020,3,1,-48.101159,159.192632,39.106061,44.132709
7,2013,6,12,-73.690842,22.783758,18.223609,28.181760
8,2017,5,26,21.309482,-41.250059,75.536141,-3.876738
9,2021,2,22,-21.156842,-174.252149,42.515587,15.635518


In [65]:
test.iloc[list(test['Year'].nlargest(25).index)]
test["Day"].notnull().sum()

10

In [15]:
import pandas as pd

# Sample data
data = pd.DataFrame({
    'latitude': [10.1, 10.2, 10.3, 10.4],
    'longitude': [20.1, 20.2, 20.3, 20.4],
    'value': [100, 200, 300, 400]
})

# Coordinate list
coordinates = [(20.1, 10.1), (20.1, 10.2), (20.2, 10.1), (20.2, 10.2)]

# Extract rows where 'latitude' and 'longitude' match the coordinates
filtered_data = data[data[['longitude', 'latitude']].apply(tuple, axis=1).isin(coordinates)]

# Display result
filtered_data['test'] = "test"
print(filtered_data)


       latitude     longitude  value  test
0 10.1000000000 20.1000000000    100  test
1 10.2000000000 20.2000000000    200  test


/var/folders/5h/rb902v091ys_0c5x0jt8sbjw0000gp/T/ipykernel_38071/767964802.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['test'] = "test"


In [49]:
import pandas as pd
import geopandas as gpd
import os
from geopy import distance
import calendar

# 设置路径和读取数据
os.chdir("/Users/anorawu/Documents/GitHub/CloudSeeding/data/炮台数据")

# 加载炮台数据
fort_data = pd.read_csv("cleaned_炮台数据.csv")

lon_max = fort_data['longitude'].max() + 1
lon_min = fort_data['longitude'].min() - 1
lat_max = fort_data['latitude'].max() + 1
lat_min = fort_data['latitude'].min() - 1

fort_points = fort_data[['longitude', 'latitude']].drop_duplicates()

# 加载气象站数据
rain_data = pd.read_stata("weatherstation_2010_2022.dta")
rain_data = rain_data.dropna(subset=['Lat', 'Lon'])
rain_data = rain_data[(rain_data['Lat'] >= lat_min) & (rain_data['Lat'] <= lat_max)]
rain_data = rain_data[(rain_data['Lon'] >= lon_min) & (rain_data['Lon'] <= lon_max)]

rain_points = rain_data[['StationId', 'Lon', 'Lat']].drop_duplicates()

# 转换为GeoDataFrame
rain_gdf = gpd.GeoDataFrame(
    rain_points,
    geometry=gpd.points_from_xy(rain_points['Lon'], rain_points['Lat']),
    crs="EPSG:4326"
)

fort_gdf = gpd.GeoDataFrame(
    fort_points,
    geometry=gpd.points_from_xy(fort_points['longitude'], fort_points['latitude']),
    crs="EPSG:4326"
)


radius = 0.5
counts = []
count_zero = 0
count_one = 0
count_two = 0
count_three = 0
count_four = 0
count_five = 0
count_enough = 0

for _, fort_point in fort_gdf.iterrows():  
    buffer = fort_point.geometry.buffer(radius)  
    
    # Correct use of 'within' and counting True values
    count = rain_gdf['geometry'].within(buffer).sum()  
    if count == 0:
        count_zero += 1
    if count == 1:
        count_one += 1
    if count == 2:
        count_two += 1
    if count == 3:
        count_three += 1
    if count == 4:
        count_four += 1
    if count == 5:
        count_five += 1
    if count > 5:
        count_enough += 1                        
    counts.append(count)
        
# Final ratio calculation
print(sum(counts) / len(fort_gdf))
print("zero:",count_zero)
print("one:",count_one)
print("two:",count_two)
print("three:",count_three)
print("four:",count_four)
print("five:",count_five)
print(">five", count_enough)
print(len(counts))


21.82320987654321
zero: 3
one: 0
two: 6
three: 15
four: 12
five: 12
>five 1977
2025


In [11]:
fort_data

,Unnamed: 0.1,Unnamed: 0,weibo_text,operation_year,operation_month,operation_day,prov,city,county,area,location,coordinate,coordinate_level,address,formatted_address,operation_date,first_operation_date,longitude,latitude
0,0,0,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,云南省西双版纳傣族自治州星火山,"100.179358,22.092365",住宅区,西双版纳傣族自治州星火山,云南省西双版纳傣族自治州勐海县星火山,2020-03-20,2020-03-20,100.179358,22.092365
1,1,1,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,云南省西双版纳傣族自治州布朗山,"100.406481,21.626384",村庄,西双版纳傣族自治州布朗山,云南省西双版纳傣族自治州勐海县布朗山,2020-03-20,2020-03-20,100.406481,21.626384
2,2,2,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,云南省西双版纳傣族自治州勐混,"100.390518,21.841447",乡镇,西双版纳傣族自治州勐混,云南省西双版纳傣族自治州勐海县勐混镇,2020-03-20,2020-03-20,100.390518,21.841447
3,4,4,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,云南省西双版纳傣族自治州勐邦水库,"100.290934,21.900714",住宅区,西双版纳傣族自治州勐邦水库,云南省西双版纳傣族自治州勐海县勐邦水库,2020-03-20,2020-03-20,100.290934,21.900714
4,5,5,【甘肃民乐：人工增雨全县普降甘霖】6月以来，我县气温特高，降水特少，最长连续无降水日达19天...,2023,8,24,甘肃省,张掖市,民乐县,海潮坝、双树寺人影作业点,甘肃省张掖市海潮坝,"100.592715,38.325645",住宅区,张掖市海潮坝,甘肃省张掖市民乐县海潮坝,2023-08-24,2023-08-24,100.592715,38.325645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2619,9559,9559,【110发火箭弹炮弹齐上天 德州打下“春雨”】一场久违的春雨让德州市的旱情得到有效缓解。从4...,2016,4,15,山东省,德州市,NaN,22个作业点,山东省德州运河开发区,"116.266042,37.447177",住宅区,德州运河开发区,山东省德州市德城区运河开发区,2016-04-15,2016-04-15,116.266042,37.447177
2620,9560,9560,【110发火箭弹炮弹齐上天 德州打下“春雨”】一场久违的春雨让德州市的旱情得到有效缓解。从4...,2016,4,15,山东省,德州市,NaN,22个作业点,山东省德州黄河涯乡,"116.346984,37.317352",乡镇,德州黄河涯乡,山东省德州市德城区黄河涯乡,2016-04-15,2016-04-15,116.346984,37.317352
2621,9561,9561,【110发火箭弹炮弹齐上天 德州打下“春雨”】一场久违的春雨让德州市的旱情得到有效缓解。从4...,2016,4,15,山东省,德州市,NaN,22个作业点,山东省德州董子巷街,"116.382575,37.444296",道路,德州董子巷街,山东省德州市德城区董子巷,2016-04-15,2016-04-15,116.382575,37.444296
2622,9562,9562,【110发火箭弹炮弹齐上天 德州打下“春雨”】一场久违的春雨让德州市的旱情得到有效缓解。从4...,2016,4,15,山东省,德州市,NaN,22个作业点,山东省德州马颊河沿岸区域,"116.421428,37.345536",住宅区,德州马颊河沿岸区域,山东省德州市德城区马颊河,2016-04-15,2016-04-15,116.421428,37.345536


In [139]:
lat_complete_temp = np.arange(18,55,0.5).tolist()
lon_complete_temp = np.arange(73.125, 135, 0.625).tolist()
# Remove useless digits
lat_complete_merra = [int(num) if num == int(num) else num for num in lat_complete_temp]
lon_complete_merra = [int(num) if num == int(num) else num for num in lon_complete_temp]
print(lon_complete_merra)

[73.125, 73.75, 74.375, 75, 75.625, 76.25, 76.875, 77.5, 78.125, 78.75, 79.375, 80, 80.625, 81.25, 81.875, 82.5, 83.125, 83.75, 84.375, 85, 85.625, 86.25, 86.875, 87.5, 88.125, 88.75, 89.375, 90, 90.625, 91.25, 91.875, 92.5, 93.125, 93.75, 94.375, 95, 95.625, 96.25, 96.875, 97.5, 98.125, 98.75, 99.375, 100, 100.625, 101.25, 101.875, 102.5, 103.125, 103.75, 104.375, 105, 105.625, 106.25, 106.875, 107.5, 108.125, 108.75, 109.375, 110, 110.625, 111.25, 111.875, 112.5, 113.125, 113.75, 114.375, 115, 115.625, 116.25, 116.875, 117.5, 118.125, 118.75, 119.375, 120, 120.625, 121.25, 121.875, 122.5, 123.125, 123.75, 124.375, 125, 125.625, 126.25, 126.875, 127.5, 128.125, 128.75, 129.375, 130, 130.625, 131.25, 131.875, 132.5, 133.125, 133.75, 134.375]


In [137]:
temp_df_id

,StationId,Lat,Lon,lat_mode,lon_mode,Alti,Year,Mon,Day,EVP,...,SSH,TEM_Avg,TEM_Max,TEM_Min,WIN_S_2mi_Avg,WIN_S_10mi_Avg,WIN_D_S_Max,WIN_S_Max,WIN_D_INST_Max,WIN_S_Inst_Max
12032012,882021.0,35.6500,98.4100,35.6500,98.4100,3728.5,2021.0,6.0,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12032013,882021.0,35.6500,98.4100,35.6500,98.4100,3728.5,2021.0,7.0,12.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12032014,882021.0,35.6500,98.4100,35.6500,98.4100,3728.5,2021.0,7.0,13.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12032015,882021.0,35.6547,98.4144,35.6547,98.4144,3728.5,2022.0,11.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12032016,882021.0,35.6547,98.4144,35.6547,98.4144,3728.5,2022.0,11.0,30.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
"""
Script Name: cloud_checking.py
Author: Wanru Wu
Date: Jan 6, 2025
Purpose: Check the completeness of cloud data 
"""

import pandas as pd
import calendar
import numpy as np


if __name__ == "__main__":
    
    ### Set directories
    data_dir = "/Users/anorawu/BFI Dropbox/Wanru Wu/Cloudseeding/data"

    # cloud thickness and fraction
    modis_dir = f"{data_dir}/MODIS"
    # weather station statistics
    weather_station_dir = f"{data_dir}/weatherstation_2010_2022.dta"
    # air, ice, and liquid
    merra_dir = f"{data_dir}/MERRA_levmean" 

    ### If true, then check the data
    check_modis = True
    check_weather_station = True
    check_merra = True

    ### Check merra data
    if check_merra:
        print("\n\n\n check merra \n\n\n")

        lat_complete_temp = np.arange(18,55,0.5).tolist()
        lon_complete_temp = np.arange(73.125, 135, 0.625).tolist()
        # Remove useless digits
        lat_complete_merra = [int(num) if num == int(num) else num for num in lat_complete_temp]
        lon_complete_merra = [int(num) if num == int(num) else num for num in lon_complete_temp]
        try:
            for year in range(2010, 2024):
                for month in range(1, 13):
                    # Get the data frame for the month and year
                    file_path = f"{merra_dir}/merra{year}{month:02}.dta"
                    temp_df   = pd.read_stata(file_path)
                    pd.set_option('display.float_format', '{:.10f}'.format)

                    # Check if dates are complete
                    days_in_month = range(1,calendar.monthrange(year, month)[1]+1)

                    # Extract all dates
                    date_list     = list(temp_df['day'].unique())

                    # Check if all dates are available
                    missing_days = [item for item in days_in_month if item not in date_list]
                    if missing_days:
                        print(f"In year {year} and month {month}, the missing days are", missing_days)
                    else:
                        print(f"In year {year} and month {month}, days are complete")

                    # Check if latitudes and longitudes are complete
                    print(temp_df['lon'].unique())
                    lon_list = [x for x in temp_df['lon'].unique()]
                    lat_list = [x for x in temp_df['lat'].unique()]


                    missing_lat = [item for item in lat_complete_merra if item not in lat_list]
                    missing_lon = [item for item in lon_complete_merra if item not in lon_list]
        except Exception as e:
            print(e)





 check merra 



In year 2010 and month 1, days are complete
[ 73.125  73.75   74.375  75.     75.625  76.25   76.875  77.5    78.125
  78.75   79.375  80.     80.625  81.25   81.875  82.5    83.125  83.75
  84.375  85.     85.625  86.25   86.875  87.5    88.125  88.75   89.375
  90.     90.625  91.25   91.875  92.5    93.125  93.75   94.375  95.
  95.625  96.25   96.875  97.5    98.125  98.75   99.375 100.    100.625
 101.25  101.875 102.5   103.125 103.75  104.375 105.    105.625 106.25
 106.875 107.5   108.125 108.75  109.375 110.    110.625 111.25  111.875
 112.5   113.125 113.75  114.375 115.    115.625 116.25  116.875 117.5
 118.125 118.75  119.375 120.    120.625 121.25  121.875 122.5   123.125
 123.75  124.375 125.    125.625 126.25  126.875 127.5   128.125 128.75
 129.375 130.    130.625 131.25  131.875 132.5   133.125 133.75  134.375
 135.   ]
In year 2010 and month 2, days are complete
[ 73.125  73.75   74.375  75.     75.625  76.25   76.875  77.5    78.125
  78.75   79.3

In [18]:
# Set directories
data_dir = r"/Users/anorawu/BFI Dropbox/Wanru Wu/Cloudseeding/data"

# cloud thickness and fraction
modis_dir = f"{data_dir}/MODIS"
output_file = f"{data_dir}/data/炮台数据/processed_cloud_data_1.csv"

# Load points data
df_pt = pd.read_csv(f"{data_dir}/炮台数据/cleaned_炮台数据.csv")

In [23]:
df_pt

,Unnamed: 0.1,Unnamed: 0,weibo_text,operation_year,operation_month,operation_day,prov,city,county,area,location,coordinate,coordinate_level,address,formatted_address,operation_date,first_operation_date,longitude,latitude
0,0,0,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,云南省西双版纳傣族自治州星火山,"100.179358,22.092365",住宅区,西双版纳傣族自治州星火山,云南省西双版纳傣族自治州勐海县星火山,2020-03-20,2020-03-20,100.1793580000,22.0923650000
1,1,1,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,云南省西双版纳傣族自治州布朗山,"100.406481,21.626384",村庄,西双版纳傣族自治州布朗山,云南省西双版纳傣族自治州勐海县布朗山,2020-03-20,2020-03-20,100.4064810000,21.6263840000
2,2,2,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,云南省西双版纳傣族自治州勐混,"100.390518,21.841447",乡镇,西双版纳傣族自治州勐混,云南省西双版纳傣族自治州勐海县勐混镇,2020-03-20,2020-03-20,100.3905180000,21.8414470000
3,4,4,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,云南省西双版纳傣族自治州勐邦水库,"100.290934,21.900714",住宅区,西双版纳傣族自治州勐邦水库,云南省西双版纳傣族自治州勐海县勐邦水库,2020-03-20,2020-03-20,100.2909340000,21.9007140000
4,5,5,【甘肃民乐：人工增雨全县普降甘霖】6月以来，我县气温特高，降水特少，最长连续无降水日达19天...,2023,8,24,甘肃省,张掖市,民乐县,海潮坝、双树寺人影作业点,甘肃省张掖市海潮坝,"100.592715,38.325645",住宅区,张掖市海潮坝,甘肃省张掖市民乐县海潮坝,2023-08-24,2023-08-24,100.5927150000,38.3256450000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2619,9559,9559,【110发火箭弹炮弹齐上天 德州打下“春雨”】一场久违的春雨让德州市的旱情得到有效缓解。从4...,2016,4,15,山东省,德州市,NaN,22个作业点,山东省德州运河开发区,"116.266042,37.447177",住宅区,德州运河开发区,山东省德州市德城区运河开发区,2016-04-15,2016-04-15,116.2660420000,37.4471770000
2620,9560,9560,【110发火箭弹炮弹齐上天 德州打下“春雨”】一场久违的春雨让德州市的旱情得到有效缓解。从4...,2016,4,15,山东省,德州市,NaN,22个作业点,山东省德州黄河涯乡,"116.346984,37.317352",乡镇,德州黄河涯乡,山东省德州市德城区黄河涯乡,2016-04-15,2016-04-15,116.3469840000,37.3173520000
2621,9561,9561,【110发火箭弹炮弹齐上天 德州打下“春雨”】一场久违的春雨让德州市的旱情得到有效缓解。从4...,2016,4,15,山东省,德州市,NaN,22个作业点,山东省德州董子巷街,"116.382575,37.444296",道路,德州董子巷街,山东省德州市德城区董子巷,2016-04-15,2016-04-15,116.3825750000,37.4442960000
2622,9562,9562,【110发火箭弹炮弹齐上天 德州打下“春雨”】一场久违的春雨让德州市的旱情得到有效缓解。从4...,2016,4,15,山东省,德州市,NaN,22个作业点,山东省德州马颊河沿岸区域,"116.421428,37.345536",住宅区,德州马颊河沿岸区域,山东省德州市德城区马颊河,2016-04-15,2016-04-15,116.4214280000,37.3455360000


In [22]:
print(len(df_pt))
print(len(df_pt[['latitude','longitude','operation_year','operation_month','operation_day']].drop_duplicates()))

2624
2624


In [26]:
import pandas as pd
import numpy as np

# Create a simple DataFrame
data = {
    "value": [10, 20, 30],
    "weight": [1, 2, 3]
}

df = pd.DataFrame(data)

print(np.average(data["value"],weights=data["weight"]))
print(140/6)

23.333333333333332
23.333333333333332


In [27]:
from multiprocessing import Pool, cpu_count
print(cpu_count())

11


In [37]:
x = (.0094259816624*0.00031956594819740493 + .0766037718492*0.00018513082147373537 + 
0*0.00014412843187886127 + .001419354807*0.00010825518228461739) / (0.00031956594819740493+0.00018513082147373537+0.00014412843187886127+0.00010825518228461739)
print(x)

0.022913807915029866


In [ ]:
import pandas as pd

### Set directories
data_dir = "/Users/anorawu/BFI Dropbox/Wanru Wu/Cloudseeding/data"

# cloud thickness and fraction
modis_dir = f"{data_dir}/MODIS"
# weather station statistics
weather_station_dir = f"{data_dir}/weatherstation_2010_2022.dta"
# air, ice, and liquid
merra_dir = f"{data_dir}/MERRA_levmean" 

### If true, then check the data
check_modis = False
check_weather_station = True
check_merra = False
check_era5 = False

### Check weather station data 
if check_weather_station:

    print("\n\n\n check weather station \n\n\n")

    # Check if there are any missing days for each StationId
    def check_continuity_with_gaps(group):
        first_date = group['Date'].min()
        full_date_range = pd.date_range(start=first_date, end=group['Date'].max())
        actual_dates = group['Date']
        missing_dates = set(full_date_range) - set(actual_dates)
        print(missing_dates)
        if not missing_dates:
            return "Continuous"
        else:
            return f"Missing Dates: {sorted(missing_dates)}"

    try:
        # Get the data frame
        temp_df = pd.read_stata(weather_station_dir)
        temp_df = temp_df[temp_df['StationId']!=999999.0]

        # # Create a Date column for easier processing
        temp_df['Date'] = pd.to_datetime(temp_df['Year', 'Mon', 'Day']])
        # temp_df = temp_df.sort_values(by=['StationId', 'Date']).reset_index(drop=True)

        # # Apply the function and summarize results
        # continuity_summary = temp_df.groupby('StationId').apply(check_continuity_with_gaps)

        # # Print the results
        # for station_id, result in continuity_summary.items():
        #     if result != "Continuous":
        #         print(f"StationId: {station_id}, Status: {result}")     
    
    except Exception as e:
        print(e)
        




 check weather station 



to assemble mappings requires at least that [year, month, day] be specified: [month] is missing


In [6]:
temp_df.columns

Index(['StationId', 'Lat', 'Lon', 'lat_mode', 'lon_mode', 'Alti', 'Year',
       'Mon', 'Day', 'EVP', 'GST_Avg', 'GST_Max', 'GST_Min', 'PRE_Max_1h',
       'PRE_Time_2008', 'PRE_Time_0820', 'PRE_Time_2020', 'PRS_Avg', 'PRS_Max',
       'PRS_Min', 'RHU_Avg', 'RHU_Min', 'SSH', 'TEM_Avg', 'TEM_Max', 'TEM_Min',
       'WIN_S_2mi_Avg', 'WIN_S_10mi_Avg', 'WIN_D_S_Max', 'WIN_S_Max',
       'WIN_D_INST_Max', 'WIN_S_Inst_Max'],
      dtype='object')

In [71]:
import pandas as pd
df = pd.read_csv("/Users/anorawu/BFI Dropbox/Wanru Wu/Cloudseeding/data/炮台数据/processed_cloud_data_1.csv")
df

,longitude,latitude,operation_year,operation_month,operation_day,first_operation_date,year,month,day,wt_cloud_optical_thickness,wt_cloud_mask_fraction
0,112.499268,35.077713,2023,2,8,2023-02-08,2011,1,1,1.887183,0.464000
1,112.499268,35.077713,2023,2,8,2023-02-08,2011,1,2,8.694333,0.972531
2,112.499268,35.077713,2023,2,8,2023-02-08,2011,1,3,8.330209,0.998417
3,112.499268,35.077713,2023,2,8,2023-02-08,2011,1,4,7.004502,0.999176
4,112.499268,35.077713,2023,2,8,2023-02-08,2011,1,5,10.417659,0.997777
...,...,...,...,...,...,...,...,...,...,...,...
10655698,119.101760,34.244468,2022,4,28,2022-04-28,2023,12,27,2.624840,0.842236
10655699,119.101760,34.244468,2022,4,28,2022-04-28,2023,12,28,11.241895,0.909532
10655700,119.101760,34.244468,2022,4,28,2022-04-28,2023,12,29,10.878973,0.998271
10655701,119.101760,34.244468,2022,4,28,2022-04-28,2023,12,30,11.530517,0.982862


In [77]:
df[['longitude','latitude','wt_cloud_optical_thickness','wt_cloud_mask_fraction','year','month','day']].drop_duplicates()

,longitude,latitude,wt_cloud_optical_thickness,wt_cloud_mask_fraction,year,month,day
0,112.499268,35.077713,1.887183,0.464000,2011,1,1
1,112.499268,35.077713,8.694333,0.972531,2011,1,2
2,112.499268,35.077713,8.330209,0.998417,2011,1,3
3,112.499268,35.077713,7.004502,0.999176,2011,1,4
4,112.499268,35.077713,10.417659,0.997777,2011,1,5
...,...,...,...,...,...,...,...
10655698,119.101760,34.244468,2.624840,0.842236,2023,12,27
10655699,119.101760,34.244468,11.241895,0.909532,2023,12,28
10655700,119.101760,34.244468,10.878973,0.998271,2023,12,29
10655701,119.101760,34.244468,11.530517,0.982862,2023,12,30


In [76]:
df[['longitude','latitude','year','month','day']].drop_duplicates()

,longitude,latitude,year,month,day
0,112.499268,35.077713,2011,1,1
1,112.499268,35.077713,2011,1,2
2,112.499268,35.077713,2011,1,3
3,112.499268,35.077713,2011,1,4
4,112.499268,35.077713,2011,1,5
...,...,...,...,...,...
10655698,119.101760,34.244468,2023,12,27
10655699,119.101760,34.244468,2023,12,28
10655700,119.101760,34.244468,2023,12,29
10655701,119.101760,34.244468,2023,12,30
